In [15]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [16]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/rf_regressor.csv')

In [17]:
### Season to test results

N = 2021

In [18]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop([
        'season', 'round',
        'podium','driver_points_from'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [19]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop([
            'season', 'round',
            'podium','driver_points_from'], axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        points += list(prediction_df.predicted_points.values)
        winners += list(prediction_df.predicted_winner.values)

        # Retrain model
        X_train = pd.concat([X_train, X_test])
        y_train = np.append(y_train, y_test)

        model.fit(X_train, y_train)
        

    return points, winners

In [20]:
parameters.head()

,model,criterion,max_features,max_depth,score
0,random_forest_regressor,squared_error,0.8,51,0.631579
1,random_forest_regressor,squared_error,0.8,5,0.578947
2,random_forest_regressor,squared_error,1.0,21,0.578947
3,random_forest_regressor,squared_error,1.0,17,0.578947
4,random_forest_regressor,squared_error,1.0,27,0.578947


In [21]:
params = parameters.iloc[0]
criterion = params.criterion
max_features = params.max_features	
max_depth = params.max_depth

params

model           random_forest_regressor
criterion                 squared_error
max_features                        0.8
max_depth                            51
score                          0.631579
Name: 0, dtype: object

In [22]:
# Random Forest Regressor

test = processed.copy()
test = test[['season', 'round', 'driver', 'starting_grid', 'podium', 'driver_points_from', 'q_delta']].query('season ==@N')

model_params = (criterion, max_features, max_depth)
model = RandomForestRegressor(criterion=criterion, max_features=max_features, max_depth=max_depth)
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [23]:
test.query('predicted_winner == 1')

,season,round,driver,starting_grid,podium,driver_points_from,q_delta,predicted_points,predicted_winner
2742,2021,1,lewis_hamilton,2,1,25.0,0.388,17.796532,1
2763,2021,2,lewis_hamilton,1,2,19.0,0.000,19.696667,1
2781,2021,3,lewis_hamilton,2,1,25.0,0.007,19.160000,1
2801,2021,4,lewis_hamilton,1,1,25.0,0.000,23.050000,1
2839,2021,5,charles_leclerc,1,20,0.0,0.000,18.890000,1
2853,2021,6,lewis_hamilton,2,15,0.0,0.000,17.070000,1
2858,2021,7,max_verstappen,1,1,26.0,0.000,20.440000,1
2876,2021,8,max_verstappen,1,1,25.0,0.000,19.130000,1
2896,2021,9,max_verstappen,1,1,26.0,0.000,20.310000,1
2917,2021,11,lewis_hamilton,1,2,18.0,0.000,21.420000,1
